In [75]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from typing import Dict, List
from torch.utils.data import Dataset, DataLoader


In [101]:
def average_mmr(elos: Dict) -> int:
    s, d = 0, 0

    for j in elos['elo']:

        if j:
            s += j
            d += 1
    return s/d


def read_data_to_xs_ys():
    matches = os.listdir(os.getcwd()+'\\DataCollection\\data\\')
    ys = []
    xs = []

    for i in matches:
        try:
            if 'zip' not in i:
                f = open(os.getcwd()+'\\DataCollection\\data\\' + i)
                data = json.load(f)
                # return data['timeline'], data['elo']

                if data['timeline'] and len(data['timeline'])> 14:
                    if data['elo']:
                        
                        rating_class = rank_classification(average_mmr(data))
                        if rating_class:
                            xs.append(data['timeline'][:15])
                            ys.append(rating_class)
        except:
            print(i, 'failed')

    return xs, ys

def rank_classification(rating: float):
    mmr_ranges = [
        [0, 112],
        [112, 227], 
        [227, 479],
        [497, 579],
        [579, 757], # bronze 4 
        [757, 963], 
        [963, 1094], 
        [1094, 1207], 
        [1207, 1308],  # silver 4 
        [1308, 1418], 
        [1418, 1528], 
        [1528, 1619],
        [1619, 1702], # gold 4 
        [1702, 1793],
        [1793, 1896], 
        [1896, 1980], 
        [1980, 2045], # plat 4 
        [2045, 2146], 
        [2146, 2255], 
        [2255, 2329], 
        [2329, 2396], # diamond 4 
        [2396, 2487], 
        [2487, 2602], 
        [2602, 2729], 
        [2729, 2893], # masters 
        [2893, 3126], # grandmaster 
        [3126, 3386] # challengers 
        
    ]
    for i in range(len(mmr_ranges)):
        if mmr_ranges[i][0] <= rating < mmr_ranges[i][1]:
            return i
    return None 

In [102]:
xs, ys = read_data_to_xs_ys()

.ipynb_checkpoints failed


In [114]:
print(len(xs), len(ys))
# torch.cuda.is_available()
# print(torch.version.cuda)

# train = [xs[:600], ys[:600]]
# valid = [xs[600:], ys[600:]]
tmp = [] 
for i in range(len(xs)):
    tmp.append((torch.tensor(xs[i]), ys[i]))
train = tmp[:600]
valid = tmp[600:]

print(train[0][0].size())
print(train[0][1])



714 714
torch.Size([15, 50])
23


In [81]:
np.array(xs[0])

array([[  554,   581,   500,     0,     0,   557,   345,   500,     0,
            0,   335,   269,   500,     0,     0,   194,   457,   500,
            0,     0,   329,   650,   500,     0,     0, 14180, 14271,
          500,     0,     0, 14176, 14506,   500,     0,     0, 14398,
        14582,   500,     0,     0, 14539, 14394,   500,     0,     0,
        14404, 14201,   500,     0,     0],
       [ 1292, 12240,   500,     0,     0,  6937,  2946,   500,     0,
            0,  8816,  4364,   500,     0,     0, 10490,  3040,   500,
            0,     0, 11669,  3339,   500,     0,     0,  7904,  7803,
          500,     0,     0, 13453, 12892,   500,     0,     0,  4506,
        11605,   500,     0,     0, 12787,  3216,   500,     0,     0,
        10880,  4484,   500,     0,     0],
       [ 2622, 12870,   612,   240,    98, 12730,  3172,   683,   404,
          233,  7154,  6855,   584,   240,     0, 13200,  2703,   942,
           84,   152, 13488,  2953,   585,    84,   158,  77

In [92]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=30, num_classes=27):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)  
        # or:
        #out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        # out: (n, 10)
        return out



# class RNN(nn.Module):
#     def __init__(self, input_size, hidden_size=64, num_classes=1):
#         super(TweetRNN, self).__init__()
        
#         self.hidden_size = hidden_size
#         self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
#         self.fc = nn.Linear(hidden_size, num_classes)
    
#     def forward(self, x):
        
#         # Forward propagate the RNN
#         out, _ = self.rnn(x)
#         # Pass the output of the last time step to the classifier
#         out = self.fc(out[:, -1, :])
#         return out


# 6*10 input 
# 64 hidden units 
# 1 class which is the elo rating 



In [105]:
class CustomDataset(Dataset):

    def __init__(self, xs, ys):
       
        self.xs = xs
        self.ys = ys
    def __len__(self):
        return len(self.xs)

    def __getitem__(self, idx):
        sample = (self.xs[idx], self.ys[idx])
        return sample

In [106]:
def get_accuracy(model, data):
    loader = torch.utils.data.DataLoader(data, batch_size=256, shuffle=True)

    model.eval() # annotate model for evaluation (why do we need to do this?)

    correct = 0
    total = 0
    for imgs, labels in loader:
      
#       imgs = imgs.cuda()
#       labels = labels.cuda()
      z = model(imgs)
      pred = z.max(1, keepdim=True)
      z = z.detach().numpy()
      z = z.argmax(axis=1)
      for i in range(len(labels)):
        if z[i] == labels[i]:
          correct += 1 
        total += 1 
    print(correct/total)
    return correct/total 

In [107]:
def training(model, train_data, valid_data, batch_size=32, weight_decay=0.0,learning_rate=0.001, num_epochs=7, max_iters=1000,
          checkpoint_path=None, momentum=0.9, save_after=100):
    
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size)
    # loss function
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),
                            lr=learning_rate,
                            weight_decay=weight_decay)
    # track learning curve
    iters, losses, train_acc, val_acc = [], [], [], []
    # training
    n = 0 # the number of iterations (for plotting)
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
#             print(imgs)
            model.train() # annotate model for training
#             model.cuda()
#             imgs = imgs.cuda()
#             labels = labels.cuda()
            out = model(imgs.float())
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            train_acc.append(get_accuracy(model, train_data)) # compute training accuracy 
            val_acc.append(get_accuracy(model, valid_data))  # compute validation accuracy

            if n % 20 == 0:
                if (checkpoint_path is not None) and n > save_after:
                    torch.save(model.state_dict(), checkpoint_path.format(n))
            if n >= max_iters:
                return iters, losses, train_acc, val_acc


            n += 1
    if (checkpoint_path is not None) and n > save_after:
        torch.save(model.state_dict(), checkpoint_path.format(n))
    return iters, losses, train_acc, val_acc

In [108]:
model = RNN(50)
# model.cuda()
# device=torch.device("cuda")


In [109]:
iters, losses, train_acc, val_acc = training(model, train, valid, batch_size=32, num_epochs=50,  max_iters=1000, learning_rate=0.001, momentum=0)


RuntimeError: expected scalar type Long but found Float